In [1]:
from sklearn.model_selection import ParameterSampler
import numpy as np
from pathlib import Path
from copy import deepcopy
import json

In [2]:
sampler = ParameterSampler(
    param_distributions={'num_samples':np.linspace(100, 200, 11),
                         'temperature':np.linspace(1, 5, 10),
                         'num_tokens_to_add': np.linspace(1, 10, 10)},
    n_iter = 500
)

In [3]:
CLF_PATH = "./logs/age/target_clf/171151-1409-gru_with_amounts/model.tar.gz"
MASKED_LM_PATH = "./presets/age/lm.model.tar.gz"
DATA_PATH = "/notebook/morozov/data/age/test.jsonl"
OUTPUT_PATH = "./results/brute_force"

In [4]:
base_config={
  "data_path": DATA_PATH,
  "output_path": OUTPUT_PATH,
  "attacker": {
    "type": "concat_sampling_fool",
    "masked_lm": {
      "type": "from_archive",
      "archive_file": MASKED_LM_PATH
    },
    "classifier": {
      "type": "from_archive",
      "archive_file": CLF_PATH
    },
    "reader": {
      "type": "from_archive",
      "archive_file": CLF_PATH
    },
    "position": "end",
    "num_tokens_to_add": 2,
    "total_amount": 5000,
    "num_samples": 100,
    "temperature": 1.5,
    "device": 0
  }
}

In [5]:
command = """PYTHONPATH=. python advsber/commands/attack.py \
             {CONFIG_PATH} \
             --samples 500"""

eval_command = """PYTHONPATH=. python advsber/commands/evaluate.py {OUTPUT_PATH} {SAVE_TO}"""

In [6]:
with open('brute_force_sf_gru_age.sh', 'w') as w:
    for i, params in enumerate(sampler):
    
        params["total_amount"] = np.random.uniform(100 * params['num_tokens_to_add'], 10000 * params['num_tokens_to_add'])
        
        exp_path = f"results/brute_force/configs/exp_{i}/"
        Path(exp_path).mkdir(parents=True, exist_ok=True)
        conf_path = f"results/brute_force/configs/exp_{i}/config.json"
        output_path = f"results/brute_force/configs/exp_{i}/output.json"
        save_to = f"results/brute_force/configs/exp_{i}/res_exp.json"
        
        config = deepcopy(base_config)
        
        for key in params.keys():
            config["attacker"][key] = params[key]
            
        config["output_path"] = output_path
        
        with open(conf_path, 'w') as f:
            json.dump(config, f)
        
        w.write(command.format(CONFIG_PATH=conf_path))
        w.write('\n')
        w.write(eval_command.format(OUTPUT_PATH=output_path, SAVE_TO=save_to))
        w.write('\n')